In [1]:
from myutils import io
evalset = io.jsonload("data/eval.json")

gpt3.5、gpt4

In [5]:
from myutils import net, gpt
import os
import openai
net.set_proxy()
api_base = "https://api.nextweb.fun/openai/v1"
api_keys = [
]
from typing import Literal
optional_models = Literal['gpt-3.5-turbo', 'gpt-3.5-turbo-16k', 'gpt-4', 'gpt-4-32k']
def call_gpt(prompt: str, model: optional_models='gpt-3.5-turbo', api_key: str=None, api_base: str=None, **kwargs) -> str:
    if api_key is not None:
        openai.api_key = api_key
    if api_base is not None:
        openai.api_base = api_base
    completion = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.01,
        **kwargs
    )
    print(completion)
    return completion.choices[0].message.content
topath = "evaluated_models/gpt4.json"
if os.path.exists(topath):
    result = io.jsonload(topath)
else:
    result = []
start = len(result)
i = 0
for unit in evalset[start:]:
    question = unit['question']
    lock = True
    while lock:
        try:
            api_key = api_keys[i]
            model_response = call_gpt(question, 'gpt-4', api_key, api_base, max_tokens=512)
            lock = False
        except Exception as e:
            import traceback
            traceback.print_exc()
            print(e)
            i = (i+1)%len(api_keys)
            break
    result.append({
        "question": question,
        "answer": model_response
    })
    io.jsondump(result, topath)
    i = (i+1)%len(api_keys)
net.unset_proxy()

子密钥余额不足，请及时充值或调整用量上限，当您在使用 gpt-4 系列模型时，请确保该密钥余额大于 327680 tokens.
choices


Traceback (most recent call last):
  File "/home/kirin/.conda/envs/wkyc/lib/python3.10/site-packages/openai/openai_object.py", line 59, in __getattr__
    return self[k]
KeyError: 'choices'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_3174548/2772328579.py", line 41, in <module>
    model_response = call_gpt(question, 'gpt-4', api_key, api_base, max_tokens=512)
  File "/tmp/ipykernel_3174548/2772328579.py", line 27, in call_gpt
    return completion.choices[0].message.content
  File "/home/kirin/.conda/envs/wkyc/lib/python3.10/site-packages/openai/openai_object.py", line 61, in __getattr__
    raise AttributeError(*err.args)
AttributeError: choices


NameError: name 'model_response' is not defined

chatglm-6b、chatglm2-6b、qwen7b、baichuan-7b

In [2]:
from tqdm import tqdm
from src.referee_model import call_glm
import os
topath = "evaluated_models/llama2-7bchat.json"
if os.path.exists(topath):
    result = io.jsonload(topath)
else:
    result = []
start = len(result)
for unit in tqdm(evalset[start:]):
    question = unit['question']
    model_response = call_glm(question)
    result.append({
        "question": question,
        "answer": model_response
    })
    io.jsondump(result, topath)

100%|██████████| 122/122 [17:45<00:00,  8.74s/it]
